In [1]:
import sys
import os
import pandas as pd
import numpy as np
from Bio import SeqIO
import re

In [2]:
dataset = 'ZCP2016'

print('Loading raw data for', dataset, '...')
data = pd.read_csv('/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/raw_data/mmc2(2).csv', header=1)
print('Raw data loaded.')
data

Loading raw data for ZCP2016 ...
Raw data loaded.


,Ratio H/L Rep 1,Ratio H/L Rep 2,Ratio H/L Rep 3,Amino acid,Charge,Reverse,Contaminant,Multiplicity,Localization prob,PEP,...,Phospho (STY) Score diffs,Position in peptide,Proteins,Positions within proteins,Leading proteins,Protein names,Gene names,Sequence window,Modified sequence,Unique identifier
0,1.02070,0.97955,0.959877,S,2,NaN,NaN,___1,1.000000,1.610000e-06,...,NaN,8,A0AVK6;E9PMT9,102;102,A0AVK6,Transcription factor E2F8,E2F8;E2F8,RSGLPEAKDCIHEHLSGDEFEKSQPSRKEKS,_DCIHEHLS(ph)GDEFEK_,UID1
1,0.99815,1.13060,0.980584,S,2,NaN,NaN,___1,1.000000,1.550000e-08,...,NaN,7,A0AVK6;E9PMT9,71;71,A0AVK6,Transcription factor E2F8,E2F8;E2F8,PWTPTANLKMLISAVSPEIRNRDQKRGLFDN,_MLISAVS(ph)PEIR_,UID4
2,NaN,1.04540,NaN,S,3,NaN,NaN,___1,0.745870,7.450240e-04,...,NaN,13,A0AVK6,357,A0AVK6,Transcription factor E2F8,E2F8,AFKWTGPEISPNTSGSSPVIHFTPSDLEVRR,_WTGPEISPNTSGS(ph)SPVIHFTPSDLEVR_,UID5
3,1.06170,1.10950,0.756315,S,3,NaN,NaN,___1,0.771478,5.810360e-04,...,NaN,14,A0AVK6,358,A0AVK6,Transcription factor E2F8,E2F8,FKWTGPEISPNTSGSSPVIHFTPSDLEVRRS,_WTGPEISPNTSGSS(ph)PVIHFTPSDLEVR_,UID6
4,0.98961,0.94271,1.027295,S,2,NaN,NaN,___1,1.000000,3.662430e-04,...,NaN,8,A0AVT1;A0AVT1-2;H0Y8S8,737;263;228,A0AVT1,Ubiquitin-like modifier-activating enzyme 6,UBA6;UBA6,FPLDIRLKDGSLFWQSPKRPPSPIKFDLNEP,_DGSLFWQS(ph)PK_,UID7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16673,0.53396,NaN,NaN,T,2,NaN,NaN,___3,0.993977,1.490000e-35,...,NaN,10,Q9UHR4,257,Q9UHR4,Brain-specific angiogenesis inhibitor 1-associ...,BAIAP2L1,MIEEIKTPASTPVSGTPQASPMIERSNVVRK,_TPAST(ph)PVSGT(ph)PQAS(ph)PMIER_,UID56421
16674,0.98512,NaN,0.907359,T,2,NaN,NaN,___3,0.845275,4.160680e-03,...,NaN,5,Q9UQ35;Q9UQ35-2,1453;1453,Q9UQ35,Serine/arginine repetitive matrix protein 2,SRRM2,RSRSGSSPGLRDGSGTPSRHSLSGSSPGMKD,_DGSGT(ph)PSRHS(ph)LSGS(ph)SPGMK_,UID56510
16675,NaN,1.15480,0.897586,T,2,NaN,NaN,___3,0.773697,2.850000e-23,...,NaN,10,Q9UQ35;Q9UQ35-2;I3L4D8;I3L182,383;383;383;287,Q9UQ35,Serine/arginine repetitive matrix protein 2,SRRM2;SRRM2;SRRM2,PLLAERHGGSPQPLATTPLSQEPVNPPSEAS,_HGGSPQPLAT(ph)TPLSQEPVNPPSEASPTR_,UID56517
16676,0.82583,0.83559,0.899362,T,3,NaN,NaN,___3,0.761376,2.850000e-23,...,NaN,11,Q9UQ35;Q9UQ35-2;I3L4D8;I3L182,384;384;384;288,Q9UQ35,Serine/arginine repetitive matrix protein 2,SRRM2;SRRM2;SRRM2,LLAERHGGSPQPLATTPLSQEPVNPPSEASP,_HGGSPQPLATT(ph)PLSQEPVNPPSEASPTR_,UID56518


In [3]:
# filter data to keep only those with localization probability >= 0.85
data = data[data['Localization prob'] >= 0.85] 
# Filtering out semi-colons from 'Amino acid', 'Positions within proteins', and 'Gene names' columns
data = data[~data['Amino acid'].str.contains(';', na=False)]
data = data[~data['Positions within proteins'].str.contains(';', na=False)]
data = data[~data['Gene names'].str.contains(';', na=False)]
data

,Ratio H/L Rep 1,Ratio H/L Rep 2,Ratio H/L Rep 3,Amino acid,Charge,Reverse,Contaminant,Multiplicity,Localization prob,PEP,...,Phospho (STY) Score diffs,Position in peptide,Proteins,Positions within proteins,Leading proteins,Protein names,Gene names,Sequence window,Modified sequence,Unique identifier
27,0.90140,NaN,NaN,S,3,NaN,NaN,___1,1.000000,7.375770e-04,...,NaN,3,A1A5D9,36,A1A5D9,Bicaudal D-related protein 2,CCDC64B,SGDEGFFPFVLERRDSFLGGGPGPEEPEDLA,_RDS(ph)FLGGGPGPEEPEDLALQLQQK_,UID30
30,1.15270,NaN,NaN,S,2,NaN,NaN,___1,1.000000,1.890000e-05,...,NaN,3,A1L020,338,A1L020,RNA-binding protein MEX3A,MEX3A,VHQPGCKPLSTFRQNSLGCIGECGVDSGFEA,_QNS(ph)LGCIGECGVDSGFEAPR_,UID33
65,1.08730,NaN,1.024947,S,2,NaN,NaN,___1,0.983037,6.450000e-08,...,NaN,7,A2RTY8,360,A2RTY8,NaN,NPAL2,IPGKQMLDKIQPDSHSLSYGTLPDGSDSTKS,_IQPDSHS(ph)LSYGTLPDGSDSTK_,UID73
66,1.17630,NaN,0.927300,S,2,NaN,NaN,___1,1.000000,2.650000e-07,...,NaN,11,A2RU67,62,A2RU67,Uncharacterized protein KIAA1467,KIAA1467,VKNGKSPLGEAPEPDSDAEVAEAAKPHLSEV,_SPLGEAPEPDS(ph)DAEVAEAAK_,UID74
75,1.08150,1.02050,1.097864,S,2,NaN,NaN,___1,0.999924,5.818470e-03,...,NaN,7,A4D1E1,877,A4D1E1,Zinc finger protein 804B,ZNF804B,MYYLNKSKRNQESLGSPHICDLGKVRPMKCN,_NQESLGS(ph)PHICDLGK_,UID83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16646,0.53396,NaN,NaN,S,2,NaN,NaN,___3,0.999999,6.320000e-36,...,NaN,14,Q9UHR4,261,Q9UHR4,Brain-specific angiogenesis inhibitor 1-associ...,BAIAP2L1,IKTPASTPVSGTPQASPMIERSNVVRKDYDT,_TPASTPVSGT(ph)PQAS(ph)PMIER_,UID51989
16658,1.14730,0.97599,0.905223,T,2,NaN,NaN,___3,0.867057,4.367290e-03,...,NaN,1,P04114,3776,P04114,Apolipoprotein B-100;Apolipoprotein B-48,APOB,CKLDFREIQIYKKLRTSSFALNLPTLPEVKF,_T(ph)SS(ph)FALNLPT(ph)LPEVK_,UID54325
16659,1.10300,0.97599,0.905223,T,2,NaN,NaN,___3,0.999995,4.367290e-03,...,NaN,10,P04114,3785,P04114,Apolipoprotein B-100;Apolipoprotein B-48,APOB,IYKKLRTSSFALNLPTLPEVKFPEVDVLTKY,_T(ph)SS(ph)FALNLPT(ph)LPEVK_,UID54326
16660,0.99775,NaN,NaN,T,2,NaN,NaN,___3,1.000000,2.990000e-06,...,NaN,5,P17275,255,P17275,Transcription factor jun-B,JUNB,KEEPQTVPEARSRDATPPVSPINMEDQERIK,_S(ph)RDAT(ph)PPVS(ph)PINMEDQER_,UID54461


In [4]:
# filter data
data['Sequence window'] = data['Sequence window'].str.replace('_', '')
data

,Ratio H/L Rep 1,Ratio H/L Rep 2,Ratio H/L Rep 3,Amino acid,Charge,Reverse,Contaminant,Multiplicity,Localization prob,PEP,...,Phospho (STY) Score diffs,Position in peptide,Proteins,Positions within proteins,Leading proteins,Protein names,Gene names,Sequence window,Modified sequence,Unique identifier
27,0.90140,NaN,NaN,S,3,NaN,NaN,___1,1.000000,7.375770e-04,...,NaN,3,A1A5D9,36,A1A5D9,Bicaudal D-related protein 2,CCDC64B,SGDEGFFPFVLERRDSFLGGGPGPEEPEDLA,_RDS(ph)FLGGGPGPEEPEDLALQLQQK_,UID30
30,1.15270,NaN,NaN,S,2,NaN,NaN,___1,1.000000,1.890000e-05,...,NaN,3,A1L020,338,A1L020,RNA-binding protein MEX3A,MEX3A,VHQPGCKPLSTFRQNSLGCIGECGVDSGFEA,_QNS(ph)LGCIGECGVDSGFEAPR_,UID33
65,1.08730,NaN,1.024947,S,2,NaN,NaN,___1,0.983037,6.450000e-08,...,NaN,7,A2RTY8,360,A2RTY8,NaN,NPAL2,IPGKQMLDKIQPDSHSLSYGTLPDGSDSTKS,_IQPDSHS(ph)LSYGTLPDGSDSTK_,UID73
66,1.17630,NaN,0.927300,S,2,NaN,NaN,___1,1.000000,2.650000e-07,...,NaN,11,A2RU67,62,A2RU67,Uncharacterized protein KIAA1467,KIAA1467,VKNGKSPLGEAPEPDSDAEVAEAAKPHLSEV,_SPLGEAPEPDS(ph)DAEVAEAAK_,UID74
75,1.08150,1.02050,1.097864,S,2,NaN,NaN,___1,0.999924,5.818470e-03,...,NaN,7,A4D1E1,877,A4D1E1,Zinc finger protein 804B,ZNF804B,MYYLNKSKRNQESLGSPHICDLGKVRPMKCN,_NQESLGS(ph)PHICDLGK_,UID83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16646,0.53396,NaN,NaN,S,2,NaN,NaN,___3,0.999999,6.320000e-36,...,NaN,14,Q9UHR4,261,Q9UHR4,Brain-specific angiogenesis inhibitor 1-associ...,BAIAP2L1,IKTPASTPVSGTPQASPMIERSNVVRKDYDT,_TPASTPVSGT(ph)PQAS(ph)PMIER_,UID51989
16658,1.14730,0.97599,0.905223,T,2,NaN,NaN,___3,0.867057,4.367290e-03,...,NaN,1,P04114,3776,P04114,Apolipoprotein B-100;Apolipoprotein B-48,APOB,CKLDFREIQIYKKLRTSSFALNLPTLPEVKF,_T(ph)SS(ph)FALNLPT(ph)LPEVK_,UID54325
16659,1.10300,0.97599,0.905223,T,2,NaN,NaN,___3,0.999995,4.367290e-03,...,NaN,10,P04114,3785,P04114,Apolipoprotein B-100;Apolipoprotein B-48,APOB,IYKKLRTSSFALNLPTLPEVKFPEVDVLTKY,_T(ph)SS(ph)FALNLPT(ph)LPEVK_,UID54326
16660,0.99775,NaN,NaN,T,2,NaN,NaN,___3,1.000000,2.990000e-06,...,NaN,5,P17275,255,P17275,Transcription factor jun-B,JUNB,KEEPQTVPEARSRDATPPVSPINMEDQERIK,_S(ph)RDAT(ph)PPVS(ph)PINMEDQER_,UID54461


In [5]:
def match_seq_to_genename(dataset, seq_column):
    '''
    Maps amino acid sequences to gene names using the loaded fasta file.
    
    args:
    =====
    dataset: <pd.Dataframe> with a column of amino acid sequences
    seq_column: <str> column name containing amino acid sequences
    
    out:
    ====
    dataset: <pd.Dataframe> with an additional column containing gene names
    '''    

    fasta_sequence = list(SeqIO.parse(open(f'/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/raw_data/UP000005640_9606.fasta'), "fasta"))
    
    
    gene_dict = {}
    
    # iterate over rows in seq_column
    for i in dataset[seq_column]:
        print(i)
        i_str = str(i)
        for seq_record in fasta_sequence:
            matches = re.findall(i_str, str(seq_record.seq))
            if matches:
                print(f"Match found for sequence: {seq_record}")
                gene_name_match = re.search(r"GN=(\w+)", seq_record.description)
                print('Gene name match:', gene_name_match)
                # gene_name_match = re.search("GN=(\w+)", seq_record.description)
                if gene_name_match:
                    gene_name = gene_name_match.group(1)
                    gene_dict[i] = gene_name
                    print(f"Match found: {i_str} -> {gene_name}")
                else: 
                    print(f"No gene name found in description for sequence: {i_str}")
    
    # map sequences to gene names           
    dataset['GeneName'] = dataset[seq_column].map(gene_dict) 
    print('Amino acid sequences matched to gene names.')
    return dataset 
    

In [6]:
data = match_seq_to_genename(data, 'Sequence window')

SGDEGFFPFVLERRDSFLGGGPGPEEPEDLA
Match found for sequence: ID: sp|A1A5D9|BICL2_HUMAN
Name: sp|A1A5D9|BICL2_HUMAN
Description: sp|A1A5D9|BICL2_HUMAN BICD family-like cargo adapter 2 OS=Homo sapiens OX=9606 GN=BICDL2 PE=1 SV=2
Number of features: 0
Seq('MSSPDGPSFPSGPLSGGASPSGDEGFFPFVLERRDSFLGGGPGPEEPEDLALQL...RRT')
Gene name match: <re.Match object; span=(79, 88), match='GN=BICDL2'>
Match found: SGDEGFFPFVLERRDSFLGGGPGPEEPEDLA -> BICDL2
VHQPGCKPLSTFRQNSLGCIGECGVDSGFEA
Match found for sequence: ID: sp|A1L020|MEX3A_HUMAN
Name: sp|A1L020|MEX3A_HUMAN
Description: sp|A1L020|MEX3A_HUMAN RNA-binding protein MEX3A OS=Homo sapiens OX=9606 GN=MEX3A PE=1 SV=1
Number of features: 0
Seq('MPSLVVSGIMERNGGFGELGCFGGSAKDRGLLEDERALQLALDQLCLLGLGEPP...IFS')
Gene name match: <re.Match object; span=(72, 80), match='GN=MEX3A'>
Match found: VHQPGCKPLSTFRQNSLGCIGECGVDSGFEA -> MEX3A
IPGKQMLDKIQPDSHSLSYGTLPDGSDSTKS
Match found for sequence: ID: sp|Q9H841|NPAL2_HUMAN
Name: sp|Q9H841|NPAL2_HUMAN
Description: sp|Q9H841

In [7]:
data['Phosphosite'] = data['Amino acid'].astype(str) + '(' + data['Positions within proteins'].astype(str) + ')'

In [8]:
keepcols = ['Phosphosite'] + ['GeneName'] + [col for col in data.columns if 'Ratio' in col]
data = data[keepcols]
data

,Phosphosite,GeneName,Ratio H/L Rep 1,Ratio H/L Rep 2,Ratio H/L Rep 3
27,S(36),BICDL2,0.90140,NaN,NaN
30,S(338),MEX3A,1.15270,NaN,NaN
65,S(360),NIPAL2,1.08730,NaN,1.024947
66,S(62),FAM234B,1.17630,NaN,0.927300
75,S(877),ZNF804B,1.08150,1.02050,1.097864
...,...,...,...,...,...
16646,S(261),BAIAP2L1,0.53396,NaN,NaN
16658,T(3776),APOB,1.14730,0.97599,0.905223
16659,T(3785),APOB,1.10300,0.97599,0.905223
16660,T(255),JUNB,0.99775,NaN,NaN


In [9]:
Ratio_columns = [col for col in data.columns if 'Ratio' in col]
data[Ratio_columns] = data[Ratio_columns].apply(pd.to_numeric, errors='coerce')

/var/folders/1v/y_z33m9d25d2tszlfrvnp2300000gn/T/ipykernel_902/881820786.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[Ratio_columns] = data[Ratio_columns].apply(pd.to_numeric, errors='coerce')


In [10]:
def log2_transform(dataset):
    '''
    Log2 transform a dataset.
    
    args:
    =====
    dataset: <pd.Dataframe>
    
    out:
    ====
    dataset: <pd.Dataframe> with log2 transformed values

    '''
    cols_to_transform = dataset[Ratio_columns]
    dataset[Ratio_columns] = cols_to_transform.apply(np.log2)
    print('Data has been log2 transformed.')
    return dataset


In [11]:
data = log2_transform(data)
print(f"DataFrame after log2 transformation:\n{data}") # Print the DataFrame after log2 transformation
data

Data has been log2 transformed.
DataFrame after log2 transformation:
      Phosphosite  GeneName  Ratio H/L Rep 1  Ratio H/L Rep 2  Ratio H/L Rep 3
27          S(36)    BICDL2        -0.149761              NaN              NaN
30         S(338)     MEX3A         0.205017              NaN              NaN
65         S(360)    NIPAL2         0.120750              NaN         0.035549
66          S(62)   FAM234B         0.234256              NaN        -0.108892
75         S(877)   ZNF804B         0.113034         0.029276         0.134699
...           ...       ...              ...              ...              ...
16646      S(261)  BAIAP2L1        -0.905196              NaN              NaN
16658     T(3776)      APOB         0.198243        -0.035062        -0.143655
16659     T(3785)      APOB         0.141433        -0.035062        -0.143655
16660      T(255)      JUNB        -0.003250              NaN              NaN
16673      T(257)  BAIAP2L1        -0.905196              NaN 

/var/folders/1v/y_z33m9d25d2tszlfrvnp2300000gn/T/ipykernel_902/1555182160.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[Ratio_columns] = cols_to_transform.apply(np.log2)


,Phosphosite,GeneName,Ratio H/L Rep 1,Ratio H/L Rep 2,Ratio H/L Rep 3
27,S(36),BICDL2,-0.149761,NaN,NaN
30,S(338),MEX3A,0.205017,NaN,NaN
65,S(360),NIPAL2,0.120750,NaN,0.035549
66,S(62),FAM234B,0.234256,NaN,-0.108892
75,S(877),ZNF804B,0.113034,0.029276,0.134699
...,...,...,...,...,...
16646,S(261),BAIAP2L1,-0.905196,NaN,NaN
16658,T(3776),APOB,0.198243,-0.035062,-0.143655
16659,T(3785),APOB,0.141433,-0.035062,-0.143655
16660,T(255),JUNB,-0.003250,NaN,NaN


In [12]:
def create_phos_ID(dataset):
    '''
    Concatenates GeneName and Phosphosite columns.
    
    args:
    =====
    dataset: <pd.Dataframe> with columns 'GeneName' and 'Phosphosite'
    
    out:
    ====
    dataset: <pd.Dataframe> with 'phosphosite_ID' column and 'GeneName' + 'Phosphosite' columns dropped
    '''
    dataset.loc[:, 'phosphosite_ID'] = dataset['GeneName'].astype(str) + '_' + dataset['Phosphosite'].astype(str)
    dataset = dataset.drop(columns=['Phosphosite', 'GeneName'])
    print('Phosphosite IDs created.')
    return dataset

data = create_phos_ID(data) # call function to create phosphosite_ID column

print('Phosphosite IDs created.')

Phosphosite IDs created.
Phosphosite IDs created.


/var/folders/1v/y_z33m9d25d2tszlfrvnp2300000gn/T/ipykernel_902/3553972964.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.loc[:, 'phosphosite_ID'] = dataset['GeneName'].astype(str) + '_' + dataset['Phosphosite'].astype(str)


In [13]:
def clean_phosID_col(data):
    data = data[~data.phosphosite_ID.str.contains('nan', case=False, na=False)]
    data = data[~data.phosphosite_ID.str.contains(';', case=False, na=False)]
    data = data[~data.phosphosite_ID.str.contains('-', case=False, na=False)]

    # Add this line to remove decimals from phosphosite_ID (e.g., S123.0 -> S123)
    data['phosphosite_ID'] = data['phosphosite_ID'].apply(lambda x: re.sub(r'\((\d+)\.0+\)', r'(\1)', x))
    
    data_grouped = data.groupby(by='phosphosite_ID')
    
    if len(data) != len(data_grouped):
        numeric_cols = data.select_dtypes(include=[np.number]).columns.tolist()
        non_numeric_cols = data.columns.difference(numeric_cols + ['phosphosite_ID']).tolist()
        data_numeric = data_grouped[numeric_cols].mean()
        data_categorical = data_grouped[non_numeric_cols].first().reset_index()
        
        # Merge numeric and non-numeric parts
        data = pd.merge(data_categorical, data_numeric, on='phosphosite_ID')
        print('Phosphosites with multiple measurements have been averaged')
    else:
        print('There are no phosphosites with multiple measurements')

    # Replace inf values with NaNs
    data = data.replace([np.inf, -np.inf], np.nan)
    
    # Ensure phosphosite_ID is first column
    if data.columns[0] != 'phosphosite_ID':
        phosphosite_ID = data.pop('phosphosite_ID')
        data.insert(0, 'phosphosite_ID', phosphosite_ID)

    return data

In [14]:
data = clean_phosID_col(data)
print("After cleaning phosphosite_ID column:")
data

Phosphosites with multiple measurements have been averaged
After cleaning phosphosite_ID column:


,phosphosite_ID,Ratio H/L Rep 1,Ratio H/L Rep 2,Ratio H/L Rep 3
0,AARS1_S(555),-0.080317,0.096667,-0.783080
1,AATF_S(153),NaN,NaN,0.022595
2,AATF_S(203),-0.009975,0.073957,0.052221
3,AATF_T(146),-0.148945,NaN,0.073772
4,AATF_T(211),NaN,NaN,0.010134
...,...,...,...,...
2728,ZNRF2_S(145),0.008917,-0.008871,-0.050328
2729,ZNRF2_S(82),-0.175696,NaN,-0.029418
2730,ZRSR2_S(349),0.078610,0.063503,-0.010923
2731,ZSCAN22_S(9),0.141694,0.021338,0.088317


In [15]:
data.to_csv(f'/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/PreprocessedDatasets/ZCP2016.csv', index=False)


print(dataset, 'has been saved to CSV successfully!', data)

ZCP2016 has been saved to CSV successfully!      phosphosite_ID  Ratio H/L Rep 1  Ratio H/L Rep 2  Ratio H/L Rep 3
0      AARS1_S(555)        -0.080317         0.096667        -0.783080
1       AATF_S(153)              NaN              NaN         0.022595
2       AATF_S(203)        -0.009975         0.073957         0.052221
3       AATF_T(146)        -0.148945              NaN         0.073772
4       AATF_T(211)              NaN              NaN         0.010134
...             ...              ...              ...              ...
2728   ZNRF2_S(145)         0.008917        -0.008871        -0.050328
2729    ZNRF2_S(82)        -0.175696              NaN        -0.029418
2730   ZRSR2_S(349)         0.078610         0.063503        -0.010923
2731   ZSCAN22_S(9)         0.141694         0.021338         0.088317
2732  ZZEF1_S(1475)        -0.139760        -0.107508        -0.185740

[2733 rows x 4 columns]
